# PREVIOUS WORK

In [2]:
# Start Python Imports
import math, time, random, datetime

# Data Manipulation
import numpy as np
import pandas as pd

# Visualization 
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
plt.style.use('seaborn-whitegrid')

# Preprocessing
from scipy.linalg import svd

#*** GET DATA AND DROP DATA ***

# Set the data equal to some variables so we can use them later
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
sample_submission = pd.read_csv('./data/sample_submission.csv')

# We remove the features Alley, FireplaceQu, PoolQC, Fence and MiscFeature from the dataset
# Because of too many missing values
train = train.drop(["Alley", "FireplaceQu", "PoolQC", "Fence", "MiscFeature"], axis=1)

#*** FILL VALUES ***

# df is used as stand-in name for train
def fill_missing_or_nan_values(df):
    fill_with = 0
    
    # Get the most common element by using size(), which returns the element and how common it is
    for column in df:
        column_size = df.groupby([column]).size()
        print(column_size)
        # Check if the column is an object, float64 or int64
        is_it_float = (df[column].dtype == np.float64)
        is_it_int = (df[column].dtype == np.int64)
        
        # If it is an object, find the most common element and fill missing and NaN values
        if(not is_it_float and not is_it_int):
            for i in column_size:
                if i > fill_with:
                    fill_with = i
                    
        # If it is either a float64 or int64, then calculate the mean and fill missing and NaN values            
        else:
            if is_it_float:
                fill_with = np.nanmean(df[column], dtype=np.float64)
            if is_it_int:
                fill_with = np.nanmean(df[column], dtype=np.int64)
        
        # Fill the values in our dataset
        train[column] = train[column].fillna(fill_with)
        fill_with = 0

fill_missing_or_nan_values(train)

Id
1       1
2       1
3       1
4       1
5       1
6       1
7       1
8       1
9       1
10      1
11      1
12      1
13      1
14      1
15      1
16      1
17      1
18      1
19      1
20      1
21      1
22      1
23      1
24      1
25      1
26      1
27      1
28      1
29      1
30      1
       ..
1431    1
1432    1
1433    1
1434    1
1435    1
1436    1
1437    1
1438    1
1439    1
1440    1
1441    1
1442    1
1443    1
1444    1
1445    1
1446    1
1447    1
1448    1
1449    1
1450    1
1451    1
1452    1
1453    1
1454    1
1455    1
1456    1
1457    1
1458    1
1459    1
1460    1
Length: 1460, dtype: int64
MSSubClass
20     536
30      69
40       4
45      12
50     144
60     299
70      60
75      16
80      58
85      20
90      52
120     87
160     63
180     10
190     30
dtype: int64
MSZoning
C (all)      10
FV           65
RH           16
RL         1151
RM          218
dtype: int64
LotFrontage
21.0     23
24.0     19
30.0      6
32.0      5
33.0     

GarageCond
Ex       2
Fa      35
Gd       9
Po       7
TA    1326
dtype: int64
PavedDrive
N      90
P      30
Y    1340
dtype: int64
WoodDeckSF
0      761
12       2
24       2
26       2
28       2
30       1
32       1
33       1
35       1
36       4
38       2
40       3
42       1
44       1
45       1
48       5
49       2
52       1
53       1
55       2
56       1
57       1
58       2
60       2
63       1
64       4
66       1
68       2
70       1
72       3
      ... 
421      1
426      1
431      2
436      1
439      1
441      1
448      1
466      1
468      1
474      1
476      1
486      2
495      1
500      1
503      1
509      1
511      1
517      1
519      1
536      1
550      1
574      1
576      1
586      1
635      1
668      1
670      1
728      1
736      1
857      1
Length: 274, dtype: int64
OpenPorchSF
0      656
4        1
8        1
10       1
11       1
12       3
15       1
16       8
17       2
18       5
20      21
21       5
22       6
23  

# RANDOM FORREST WORK